<a href="https://colab.research.google.com/github/Savi0301/Statistical-Data-Analysis-on-Air-Pollution/blob/main/StatisticalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
import os
from netCDF4 import Dataset
from osgeo import gdal
from datetime import date

# PM file path
# pmfile = '/V5GL02.HybridPM25.Global.201101-201112.nc'
pmfile = '/content/V5GL02.HybridPM25.Global.201101-201112.nc'
maskfile   = '/content/cities_13k_largerext.tif' # mask data
lookupfile = '/content/Revised_GHS-SMOD HDC_Lookup_DAM_112421.csv' # infor about the cities


IIPAR     = 91
JJPAR     = 89
dmax      = 396
k_pm      = 34
k_o3      = 62
k_no      = 34
ks        = [k_pm,k_o3,k_no]

di        = (date.toordinal(date(2010, 11, 30))) #matlab starts from 0000 python from 0001
pollutant = ['PM','O3','NO']
pmax      = len(pollutant)
# emission factors
SPD        = 60 * 60 * 24
CM2M       = 10000
MW         = [46.005, 12.0107+15.9994,12.0107]
KGCONV     = np.array(MW) * 0.001 / (6.0221415E23)
efactor    = SPD * CM2M * KGCONV
# health information
ages       = [25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
ageids     = [10,11,12,13,14,15,16,17,18,19,20,30,31,32,235]
amax       = len(ages)
# mortality rate outcome names
mortid      = [976,509,426,322,494,493]
mmax        = len(mortid)


# Initializations
# Sensitivities
PMGDT = np.zeros((IIPAR,JJPAR,dmax,k_pm))
PMEMS = PMGDT
O3GDT = np.zeros((IIPAR,JJPAR,dmax,k_o3))
O3EMS = O3GDT
NOGDT = np.zeros((IIPAR,JJPAR,dmax,k_no))
NOEMS = NOGDT

# Other
inames    = np.zeros((62,3))
O3        = np.zeros((91,89))
CT = np.zeros((91,89))

inames0= []
inames1= []
inames2= []

# Initializations for PM
PMGDT = np.zeros((IIPAR, JJPAR, dmax, k_pm))
PMEMS = PMGDT

# Read in PM Data
#-------------------------------------------------------------------------%
PM = Dataset(pmfile, mode='r').variables['GWRPM25'][:]
latpm = Dataset(pmfile, mode='r').variables['lat'][:]
lonpm = Dataset(pmfile, mode='r').variables['lon'][:]
PM = np.transpose(PM)
#-------------------------------------------------------------------------%

for p  in range(pmax):
    A   = np.zeros((IIPAR,JJPAR,dmax,ks[p])) #######################################
    G = np.zeros((IIPAR,JJPAR,dmax,ks[p]))#######################################
    TOT = np.zeros((ks[p])) #######################################
    C = 0

dataset = gdal.Open(maskfile, gdal.GA_ReadOnly)#gdal.GA_ReadOnly
for x in range(1, dataset.RasterCount + 1):
    band = dataset.GetRasterBand(x)
    A = band.ReadAsArray()

df = pd.read_csv(lookupfile,skiprows=1, names=['ID_HDC_G0','City_Corrected','Cluster_Corrected','Country_Corrected','GBD_Region_Corrected'])
T=np.squeeze([[df.ID_HDC_G0],[df.City_Corrected],[df.Cluster_Corrected],[df.Country_Corrected],[df.GBD_Region_Corrected]])
for ii in range(len(T[2,:])):
    if T[1,ii] == str(NAME):
        ind=ii+1

A[A!=ind] = 0
latf = np.linspace(75,-56,15720)
lonf = np.linspace(-179.5000,179.0000,43020)
ymin=np.min(np.where(np.sum(A,axis=1)>0))
ymax=np.max(np.where(np.sum(A,axis=1)>0))
xmin=np.min(np.where(np.sum(A,axis=0)>0))
xmax=np.max(np.where(np.sum(A,axis=0)>0))
# limit to just the city boundaries
Z  = A[ymin:ymax+1,xmin:xmax+1]
la = latf[ymin:ymax+1]
lo = lonf[xmin:xmax+1]



# Calculation for PM 2.5
# % PM 2.5
NUM = 0; DEN = 0
for i in range(len(lo)):
    for j in range(len(la)):
        x = ((abs(lonpm - lo[i]))); x = np.where(x == np.min(x))  # PM calculation for the city only
        y = ((abs(latpm - la[j]))); y = np.where(y == np.min(y))
        xp = ((abs(lonpo - lo[i]))); xp = np.where(xp == np.min(xp))  # for the population over the city only
        yp = ((abs(latpo - la[j]))); yp = np.where(yp == np.min(yp))
        if np.isnan(PM[x, y]) == False and POP[yp, xp] > 0:
            NUM = NUM + (np.double(PM[x, y]) * np.double(Z[j, i]) * np.double(POP[yp, xp]))  # Z is the mask and calculation using pop data and O3
            DEN = DEN + (np.double(Z[j, i]) * np.double(POP[yp, xp]))
CFPM = NUM / DEN

# PM Health Impact
# % PM2.5
# %-------------------------------------------------------------------------%
dJPMD = np.zeros((IIPAR, JJPAR, dmax, k_pm))
dJPMD_lb = np.zeros((IIPAR, JJPAR, dmax, k_pm))
dJPMD_ub = np.zeros((IIPAR, JJPAR, dmax, k_pm))
AVAIL = np.zeros(np.shape(dJ_PM)[3])

for ij in range(np.shape(dJ_PM)[3]):
    AVAIL[ij] = (np.sum(dJ_PM[:, :, :, ij]))

# More code here for calculating health impacts related to PM...
# Note: The health impact calculation section is lengthy and involves calculations based on PM data.
# It includes loops over age groups and health outcomes, and calculations of relative risks and impacts.

# PM Emission Scaling
# % PM2.5
sf = CFPM / np.sum(PMGDT)
PMEMS = PMEMS * sf

# More code may follow, focusing on the handling and processing of PM data.


NameError: ignored